<a href="https://colab.research.google.com/github/arafMustavi/Chatbot/blob/master/PythonChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setup of Environment and Required Package Loading

In [1]:
import numpy as np
import nltk
import random
import json
import json
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Tokenizer and Stemmer

In [2]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, words):
    sentence_words = [stem(word) for word in tokenized_sentence]
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words: 
            bag[idx] = 1
    return bag

The Model

In [3]:
import torch.nn as nn


class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.l2 = nn.Linear(hidden_size, hidden_size) 
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    

    def forward(self, x):
        out1 = self.l1(x)
        out1 = self.relu(out1)
        out2 = self.l2(out1)
        out2 = self.relu(out2)
        out3 = self.l3(out2)
        return out3

Dataset Load

In [4]:
with open('intents.json', 'r') as f:
    intents = json.load(f)

all_words = []
tags = []
xy = []

for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))

ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))

X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

# Hyper-parameters 
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)

class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

dataset = ChatDataset()

54 7


Train Model

In [5]:
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        print("Outputs " + outputs.type())
        print("Labels " + labels.type())

        loss = criterion(outputs, labels.long())


        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


print(f'final loss: {loss.item():.4f}')

data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')

Streaming output truncated to the last 5000 lines.
Outputs torch.FloatTensor
Labels torch.LongTensor
Outputs torch.FloatTensor
Labels torch.LongTensor
Outputs torch.FloatTensor
Labels torch.LongTensor
Outputs torch.FloatTensor
Labels torch.LongTensor
Outputs torch.FloatTensor
Labels torch.LongTensor
Outputs torch.FloatTensor
Labels torch.LongTensor
Outputs torch.FloatTensor
Labels torch.LongTensor
Outputs torch.FloatTensor
Labels torch.LongTensor
Outputs torch.FloatTensor
Labels torch.LongTensor
Outputs torch.FloatTensor
Labels torch.LongTensor
Outputs torch.FloatTensor
Labels torch.LongTensor
Outputs torch.FloatTensor
Labels torch.LongTensor
Outputs torch.FloatTensor
Labels torch.LongTensor
Outputs torch.FloatTensor
Labels torch.LongTensor
Outputs torch.FloatTensor
Labels torch.LongTensor
Outputs torch.FloatTensor
Labels torch.LongTensor
Outputs torch.FloatTensor
Labels torch.LongTensor
Outputs torch.FloatTensor
Labels torch.LongTensor
Outputs torch.FloatTensor
Labels torch.LongTensor

Bot Function

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('intents.json', 'r') as json_data:
    intents = json.load(json_data)

FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

bot_name = "ArafMUSTAVI"
print("Let's chat! (type 'quit' to exit)")
while True:
    sentence = input("You: ")
    if sentence == "quit":
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])} \n")
    else:
        print(f"{bot_name}: I do not understand...")

Let's chat! (type 'quit' to exit)
You: Hi
ArafMUSTAVI: Hey :-) 

You: I need help
ArafMUSTAVI: I do not understand...
You: Can you help me?
ArafMUSTAVI: I do not understand...
You: what do you sell?
ArafMUSTAVI: We have coffee and tea 

You: when I can get that?
ArafMUSTAVI: I do not understand...
You: quit
